# Notebook: Verifica e test delle funzioni UI in `src/lib/ui.ts`

Questo notebook guida il testing manuale e automatico delle utility UI (`showQrOverlay`, `showConfirmModal`, `escapeHtml`) e specifica come usare gli elementi grafici della libreria `nes.css` quando disponibili.

---

**Obiettivo:** fornire istruzioni step-by-step per integrare `nes.css`, compilare `src/lib/ui.ts`, eseguire test unitari ed end-to-end, e assicurare che i componenti usino le classi `nes-btn` e varianti (`is-primary`, `is-warning`, `is-danger`) dove appropriato.

## 1) Importare dipendenze e NES.css

- Opzione CDN (rapida per test manuale): aggiungi questo tag `<link>` nella tua pagina HTML di test inside `<head>`:

```html
<link rel="stylesheet" href="https://unpkg.com/nes.css/css/nes.min.css">
```

- Opzione bundle (prodotto): installa la libreria e importa nel tuo entry:

```bash
npm install nes.css
```

E poi nel tuo JS/TS entry (es. `src/main.ts`):

```ts
import 'nes.css/css/nes.min.css'
```

- Linea guida: usare le classi `nes-btn`, `is-primary`, `is-warning`, `is-danger` per pulsanti, `is-dark` per elementi scuri, e `is-centered` per allineamenti quando disponibili.

Esempio markup (usare sempre classi NES quando possibile):

```html
<button class="nes-btn is-primary" aria-label="Conferma">Conferma</button>
<button class="nes-btn is-warning" aria-label="Annulla">Annulla</button>
```

## 2) Configurazione TypeScript e build

Esempio `tsconfig.json` minimo per compilare `src/lib/ui.ts`:

```json
{
  "compilerOptions": {
    "target": "ES2020",
    "module": "ESNext",
    "declaration": false,
    "moduleResolution": "node",
    "strict": true,
    "esModuleInterop": true,
    "skipLibCheck": true,
    "outDir": "dist"
  },
  "include": ["src"]
}
```

Installare TypeScript ed esbuild (o usare tsc):

```bash
npm install --save-dev typescript esbuild
npx tsc --noEmit
# bundle con esbuild
npx esbuild src/lib/ui.ts --bundle --outfile=dist/ui.js --platform=browser
```

Nota: `esbuild` è veloce per bundle di test; per progetti più grandi considera Rollup o Vite.

## 3) Eseguire in pagina: esempio minimo per `showQrOverlay`

Crea una pagina di test `test/index.html` che include `nes.css` (via CDN o bundle) e `dist/ui.js`:

```html
<!doctype html>
<html>
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <link rel="stylesheet" href="https://unpkg.com/nes.css/css/nes.min.css">
  <script type="module" src="./dist/ui.js"></script>
</head>
<body>
  <h1>UI lib test</h1>
  <script type="module">
    import { showQrOverlay } from './dist/ui.js'
    // esempio dataUrl: crea una canvas e genera dataURL
    const c = document.createElement('canvas')
    c.width = 128; c.height = 128
    const ctx = c.getContext('2d')
    if (ctx) {
      ctx.fillStyle = '#86257b'
      ctx.fillRect(0,0,128,128)
      ctx.fillStyle = '#fff'
      ctx.fillRect(32,32,64,64)
    }
    const dataUrl = c.toDataURL('image/png')
    const profile = { userId: 'u1', displayName: 'Mario Rossi' }
    // chiamata di test
    window.showQrTest = () => showQrOverlay(dataUrl, profile)
  </script>
  <button class="nes-btn is-primary" onclick="window.showQrTest()">Mostra QR</button>
</body>
</html>
```

- Apri `test/index.html` nel browser (puoi usare un semplice static server: `npx serve test` o `npx http-server test`).
- Verifica che i bottoni nell'overlay abbiano la classe `nes-btn` e le varianti corrette (is-primary/is-warning/is-danger). Se il tuo bundle non include `nes.css`, usa il CDN link nel `<head>`.

## 4) Eseguire in pagina: esempio minimo per `showConfirmModal`

Aggiungi uno script di test nella stessa pagina o in una nuova pagina:

```html
<script type="module">
  import { showConfirmModal } from './dist/ui.js'
  document.querySelector('#confirmTest')?.addEventListener('click', async () => {
    const result = await showConfirmModal('Eliminare elemento?', 'Elimina')
    console.log('Confirm result:', result)
  })
</script>
<button id="confirmTest" class="nes-btn is-warning">Apri confirm</button>
```

- Verifica che il popup mostri due pulsanti con classi `nes-btn is-primary` e `nes-btn is-warning`.
- Controlla che la Promise risolva `true` su conferma e `false` su annulla.

## 5) Unit test per `escapeHtml` e funzioni DOM

Esempio con Vitest (o Jest):

Installare dipendenze:

```bash
npm install --save-dev vitest jsdom @types/jsdom
```

Esempio test `test/ui.spec.ts`:

```ts
import { describe, it, expect, beforeEach, afterEach } from 'vitest'
import { escapeHtml, showQrOverlay } from '../src/lib/ui'

describe('escapeHtml', () => {
  it('escapes special chars', () => {
    expect(escapeHtml('<&>"\''))
      .toBe('&lt;&amp;&gt;&quot;&#39;')
  })
})

// DOM tests (jsdom)
describe('DOM helpers', () => {
  beforeEach(() => { document.body.innerHTML = '' })
  it('showQrOverlay adds overlay with buttons', () => {
    showQrOverlay('data:image/png;base64,AAA', { userId: 'u1', displayName: 'A' })
    const overlay = document.body.querySelector('div')
    expect(overlay).not.toBeNull()
    expect(document.querySelector('#download')).not.toBeNull()
    expect(document.querySelector('#copy')).not.toBeNull()
    expect(document.querySelector('#close')).not.toBeNull()
  })
})
```

Esegui i test con:

```bash
npx vitest run
```


## 6) Test end-to-end con Puppeteer

Esempio di script E2E `test/e2e/pw-run.js` (usa Puppeteer o Playwright):

```js
const puppeteer = require('puppeteer')
const serve = require('serve')
;(async () => {
  const server = serve('test', { port: 5000, silent: true })
  const browser = await puppeteer.launch({ headless: true })
  const page = await browser.newPage()
  await page.goto('http://localhost:5000')

  // trigger overlay
  await page.click('.nes-btn')
  await page.waitForSelector('#download')

  // mock clipboard
  await page.evaluate(() => {
    navigator.clipboard = { writeText: async () => true }
  })

  await page.click('#copy')
  // click close
  await page.click('#close')
  const overlay = await page.$('#close')
  // overlay should be removed
  // close browser
  await browser.close()
  server.stop()
})()
```

Esegui:

```bash
node test/e2e/pw-run.js
```

Assicurati di mockare `navigator.clipboard` se l'ambiente non lo supporta.

## 7) Script npm e comandi per VS Code

Aggiungi gli script utili in `package.json`:

```json
"scripts": {
  "build": "esbuild src/lib/ui.ts --bundle --outfile=dist/ui.js --platform=browser",
  "test": "vitest",
  "e2e": "node test/e2e/pw-run.js",
  "serve:test": "serve test -p 5000"
}
```

Esecuzione in VS Code (Integrated Terminal): apri il terminale integrato e lancia:

```powershell
npm run build
npm run serve:test
npm run test
npm run e2e
```

Output dei comandi appare nel pannello Terminal; per i test, usa il pannello Output > Vitest se configurato.